# OBJECTIVE:
To Convert data into a form such that it can be fed into model fit function. This code only takes out the WIFI data

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import glob
from main_io_comp_vis_all import calibrate_magnetic_wifi_ibeacon_to_position, read_data_file

In [ ]:
train_paths = glob.glob('../input/indoor-location-navigation/train/*/*/*')
test_paths = glob.glob('../input/indoor-location-navigation/test/*')
mall_list = os.listdir('../input/indoor-location-navigation/train')
floor_txt_num = {'F1':1,'1F':1,'L1':1,'1L':1,'F2':3,'2F':3,'L2':3,'2L':3,'F3':4,'3F':4,'L3':4,'3L':4,'F4':5,'4F':5,'L4':5,'4L':5,
                'F5':6,'5F':6,'L5':6,'5L':6,'F6':7,'6F':7,'L6':7,'6L':7,'F7':8,'7F':8,'L7':8,'7L':8,'F8':9,'8F':9,'L8':9,'8L':9,
                'F9':10,'9F':10,'L9':10,'9L':10,'F10':11,'10F':11,'L10':11,'10L':11,'F11':12,'11F':12,'L11':12,'11L':12,
                'B1':1,'1B':1,"B":1,'B2':2,'2B':2,'B3':3,'3B':3,'B4':4,'4B':4}

# 1) Initialize DF & Create DF

In [ ]:
for mall_num, mall in enumerate(mall_list[22:51]):
    if not(os.path.exists('../input/bssid-maxfxy-unique/'+mall+'_bssid.csv')):
        # print(mall_num+20)
        continue
    bssid_list = pd.read_csv('../input/bssid-maxfxy-unique/'+mall+'_bssid.csv')
    # print(bssid_list.shape)
    #if ('0' not in list(bssid_list.columns)) or (len(bssid_list['0']==0)):
    if (bssid_list.shape[0]<2):
        # print(mall_num+20)
        continue
    mall_path = '../input/indoor-location-navigation/train/'+mall
    floors_mall = list(os.walk(mall_path))[0][1]
    bssid_col_count = len(bssid_list['0'])
    bssid_row_count=0
    mall_np = np.ones((1,bssid_col_count+3))*(-100)
    for floor in floors_mall:
        print(mall_num+22, "out of 204 , floor:",floor)   
        if floor not in list(floor_txt_num.keys()):
            continue
        floor_path = mall_path + '/'+floor+'/*'
        floor_files_paths = glob.glob(floor_path)        
        mwi_data = calibrate_magnetic_wifi_ibeacon_to_position(floor_files_paths)
        print(mall_num+22, "out of 204 , floor:",floor,' - DataRead')   
        bssid_row_count=len(list(mwi_data.keys()))
        mall_np_floor = np.ones((bssid_row_count,bssid_col_count+3))*(-100)
        idxf = 0
        for x_y in list(mwi_data.keys()):
            mwi_data_xy_wifi = pd.DataFrame(mwi_data[x_y]['wifi'])
            mwi_data_xy_wifi.columns = ['Time', 'SSID', 'BSSID', 'RSSI', 'LastTime']
            mwi_data_xy_wifi_grouped = mwi_data_xy_wifi[["BSSID", "RSSI"]].astype({"BSSID":str,"RSSI":float}).groupby(["BSSID"]).mean()
            mall_np_floor[idxf, 0], mall_np_floor[idxf, 1], mall_np_floor[idxf, 2] = floor_txt_num[floor], x_y[0], x_y[1]
            for idx, row in mwi_data_xy_wifi_grouped.iterrows():
                try:
                    bssid_idx = bssid_list[bssid_list['0']==idx].index.values[0]
                    mall_np_floor[idxf, int(bssid_idx+3)] = int(row["RSSI"])
                except:
                    continue
            print(mall_num+21, "out of 204 , floor:",floor," File:",idxf, " out of ", bssid_row_count)   
            idxf+=1        
        mall_np = np.concatenate((mall_np, mall_np_floor), axis=0)
    mall_df=pd.DataFrame(data=mall_np[1:,:], columns =['floor','x','y']+list(bssid_list['0']))
    mall_df.to_csv(mall+'_df.csv')